# SSD for battery

In [0]:
driveDir = '/content/drive'
from google.colab import drive
drive.mount(driveDir)
driveDir = driveDir + '/My Drive/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

#ssdDir = driveDir + 'NN/ssd_keras/'
ssdDir = driveDir + 'Centria Univ./SSD/'
os.chdir(ssdDir)

In [0]:
import sys
if not 'scripts' in sys.path:
  sys.path.append('scripts') 

In [0]:
!pip install keras==1.2.2
!pip install imageio
!pip install matplotlib 
!pip install pillow
!pip install h5py
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pyocr

    100% |████████████████████████████████| 184kB 5.4MB/s 
  Running setup.py bdist_wheel for keras ... - \ done
  Stored in directory: /root/.cache/pip/wheels/55/07/cf/b32db0a8d243b2fd6759d5d7cb650aa20670b2b740209cbf7e
Successfully built keras
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4
    100% |████████████████████████████████| 3.3MB 7.5MB/s 
  Running setup.py bdist_wheel for imageio ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/e0/43/31/605de9372ceaf657f152d3d5e82f42cf265d81db8bbe63cde1
Successfully built imageio
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fontconfig libcairo2 libdatrie1 libgif7 libjbig0 liblept5 libopenjp2-7
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpixman-1-0
  libtesseract4 libthai-data libthai0 libtiff5 libwebp6 libxcb-render0
  libxcb-shm

## Load model

In [0]:
import cv2
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.vgg16 import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from imageio import imread
import tensorflow as tf

from ssd import SSD300
from ssd_utils import BBoxUtility

import time

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'

np.set_printoptions(suppress=True)

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.45
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [0]:
voc_classes = ['keyword']
NUM_CLASSES = len(voc_classes) + 1

In [0]:
input_shape=(300, 300, 3)
model = SSD300(input_shape, num_classes=NUM_CLASSES)
model.load_weights('param_org4.hdf5', by_name=True)
bbox_util = BBoxUtility(NUM_CLASSES)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


## Result of keyword detection

In [0]:
import glob

path_lst = glob.glob(ssdDir + 'Battery_Image/*/*.jpeg')

In [0]:
for i in range(len(path_lst)):
  inputs = []
  images = []
  img_path = path_lst[i]
  #img_path = path_prefix + name
  img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
  img = image.img_to_array(img)
  images.append(imread(img_path))
  inputs.append(img.copy())
  inputs = preprocess_input(np.array(inputs))
  
  preds = model.predict(inputs, batch_size=1, verbose=1)
  results = bbox_util.detection_out(preds)
  
  for i, img in enumerate(images):
    # Parse the outputs.
    det_label = results[i][:, 0]
    det_conf = results[i][:, 1]
    det_xmin = results[i][:, 2]
    det_ymin = results[i][:, 3]
    det_xmax = results[i][:, 4]
    det_ymax = results[i][:, 5]

    # Get detections with confidence higher than 0.4.
    top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.08][:1]
    
    #top_indices = [0]

    top_conf = det_conf[top_indices]
    top_label_indices = det_label[top_indices].tolist()
    top_xmin = det_xmin[top_indices]
    top_ymin = det_ymin[top_indices]
    top_xmax = det_xmax[top_indices]
    top_ymax = det_ymax[top_indices]

    colors = plt.cm.hsv(np.linspace(0, 1, NUM_CLASSES)).tolist()

    plt.imshow(img / 255.)
    currentAxis = plt.gca()

    for i in range(top_conf.shape[0]):
        xmin = int(round(top_xmin[i] * img.shape[1]))
        ymin = int(round(top_ymin[i] * img.shape[0]))
        xmax = int(round(top_xmax[i] * img.shape[1]))
        ymax = int(round(top_ymax[i] * img.shape[0]))
        score = top_conf[i]
        label = int(top_label_indices[i])
        label_name = voc_classes[label - 1]
        display_txt = '{:0.2f}, {}'.format(score, label_name)
        coords = (xmin, ymin), xmax-xmin+1, ymax-ymin+1
        color = colors[label]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})
    
    plt.show()

## Test

In [0]:
from PIL import Image
import os
import sys
import cv2
import numpy as np
import pyocr
import pyocr.builders
import glob

In [0]:
# keyword list
alk = ("ALKALINE", [["LR03", "LR6", "LR14", "LR20"], ["ALKALINE", "ALK", "1.5V", "1,5V"]])
nicd = ("NICD", [["KR03", "KR6", "KR14", "KR20"], ["NI-CD", "NICD"]])
nimh = ("NIMH", [["HR03", "HR6", "HR14", "HR20"], ["NI-MH", "MH-NI", "NIMH", "MHNI"]])
liion = ("LIION", [[], ["LI-ION", "LIION", "LITHIUM", "3,7V", "3.7V"]])

klsts = [alk, nicd, nimh, liion]
k_len = len(klsts)

In [0]:
# unknonwn
UNKNOWN = "UNKNOWN"

In [0]:
# Text recognition with rotation
def txt_rcg_rotate (im, degree):
    tools = pyocr.get_available_tools()
    tool = tools[0]
    txt = tool.image_to_string(
            Image.open(im).rotate(degree),
            lang="eng",
            builder=pyocr.builders.TextBuilder(tesseract_layout=3))
    return txt.upper()

In [0]:
# Judgement label
def judge(txt):
    res = [[0] * k_len] * 2
    for i in range(k_len):
        label, lst = klsts[i]
        for j in range(len(lst)):
            for k in lst[j]:
                if k in txt:
                    res[j][i] = 1
    return res

In [0]:
# Make experiment list
def make_elst():
    elst = []
    for i in range(k_len):
        jlst = [0] * k_len
        label, lst = klsts[i]
        jlst[i] = 1
        elst.append((label, jlst))
    return elst

In [0]:
# Return label
def labeling(res):
    elst = make_elst()
    slst, wlst = res
    r = [UNKNOWN, UNKNOWN]
    for i in range(k_len):
        label, jlst = elst[i]
        if slst == jlst:
            r[0] = label
        if wlst == jlst:
            r[1] = label
    if r[0] != UNKNOWN:
        return r[0]
    else:
        return r[1]

In [0]:
dir_lst = glob.glob(ssdDir + 'Battery_Image/*')
path_lst = []

for dir in dir_lst:
  lst = glob.glob(dir + '/*.jpeg')
  path_lst.append(lst)
  
print(len(dir_lst))

37


In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.08

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.1

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.15

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9985
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.2

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9985
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9987
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9984
1/1 [==============================] - 0s
NIMH

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.25

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9985
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9987
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9984
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9991
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9992
1/1 [==============================] - 0s
UNKNOWN
1/1 [==========================

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.3

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9985
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9987
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9984
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9991
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9992
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centri

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.35

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9985
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9987
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9984
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9991
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9992
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9983
1/1 [==========

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.4

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9985
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9987
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9984
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9991
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9992
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9983
1/1 [==============================] - 0s
UNKNOWN
1/1 [==========

In [0]:
# main
counter_text = 0
counter_unknown = 0
counter_label = 0
result = []

start = time.time()
th = 0.5

for j in range(len(dir_lst)):
  print(dir_lst[j])
  counter = 0
  counter_text0 = 0
  l = len(path_lst[j])
  for img_path in path_lst[j]:
    inputs = []
    images = []
    if os.path.isfile(img_path): 
      counter += 1
      img = image.load_img(img_path, target_size=(300, 300))
      img = image.img_to_array(img)
      images.append(imread(img_path))
      inputs.append(img.copy())
      inputs = preprocess_input(np.array(inputs))
      preds = model.predict(inputs, batch_size=1, verbose=1)
      results = bbox_util.detection_out(preds)
      # Parse the outputs.
      det_conf = results[0][:, 1]
      # Get detections with confidence higher than 0.3
      top_indices = [i for i, conf in enumerate(det_conf) if conf >= th]
      if top_indices != []:
        counter_text += 1
        counter_text0 += 1
        txt = ""
        for degree in [0, 270, 90, 180]:
          txt = txt + txt_rcg_rotate(img_path, degree)
        label = labeling(judge(txt))
        print(label)
        if label != UNKNOWN:
          counter_label += 1
          result.append((dir_lst[j], label, counter_text0))
          break
        else:
          counter_unknown += 1
      if counter == l:
            result.append((dir_lst[j], label, counter_text0))
      
print("=== END ===")
print("Time: " + str(time.time() - start) + " sec")
print("Threshold: " + str(th))
print("Text Recognition: " + str(counter_text) + " images")
print("Success: " + str(counter_label))
print("Unknown: " + str(counter_unknown))
print("=== Battery ===")
for name, label, num in result:
  print("Battery: " + name)
  print("Label: " + label)
  print("Number: " + str(num))

/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9988
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9985
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9987
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9984
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9991
1/1 [==============================] - 0s
NIMH
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9992
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
ALKALINE
/content/drive/My Drive/Centria Univ./SSD/Battery_Image/IMG_9983
1/1 [==============================] - 0s
UNKNOWN
1/1 [==============================] - 0s
UNKNOWN
1/1 [==========